# Physics-Informed Neural Networks (PINNs) in PyTorch: Familiar Examples Tutorial

Absolutely! Great idea—students often find electrical circuits and heat transfer more intuitive than fluid dynamics. We'll adapt the PINN framework to two classic problems they're likely familiar with from intro physics/engineering courses:

1. **Kirchhoff's Voltage Law (KVL) for an RC Circuit**: This leads to a simple **ordinary differential equation (ODE)** for the capacitor voltage decay. It's a 1D time-domain problem.
2. **1D Heat Diffusion Equation**: The classic **heat equation**, a linear PDE modeling temperature evolution in a rod.

These showcase PINNs for both ODEs (easier entry) and PDEs. We'll keep the structure similar to the Burgers' tutorial: PyTorch-based, with automatic differentiation, Adam + LBFGS optimization, and visualizations. Each section is self-contained—run them independently in a Jupyter notebook.

Assumptions: Students know basic PyTorch, ODEs/PDEs, and circuits/heat transfer. Runtime: ~1-2 minutes per example on CPU.

## Quick PINN Recap
PINNs approximate solutions $ u(\mathbf{x}) $ (e.g., voltage or temperature) with a neural net, trained to minimize:
- **Data loss**: Matches initial/boundary conditions.
- **Physics loss**: PDE/ODE residual ≈ 0 at random "collocation" points.

## Example 1: RC Circuit ODE (Kirchhoff's Law)
### Problem Setup
Consider a series RC circuit with resistor $ R $ and capacitor $ C $, charged to initial voltage $ V_0 = 1 $ V, then discharged (switch opens at $ t=0 $).

By KVL: Voltage drop across R + C = 0 (no source), so:
\[
\frac{dV}{dt} = -\frac{V}{RC} = -\frac{V}{\tau}
\]
- Time domain: $ t \in [0, 5\tau] $, where $ \tau = RC = 1 $ s (normalize for simplicity).
- Initial condition (IC): $ V(0) = 1 $.
- Exact solution: $ V(t) = e^{-t/\tau} $ (exponential decay).

We'll use:
- **50 IC points** (just t=0, but duplicated for stability; treat as "data").
- **5,000 collocation points** in time for ODE residual.

This teaches ODE solving—PINNs shine for stiff/nonlinear ODEs, but works great here.

### Step 1: Imports and Setup

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Problem parameters
tau = 1.0  # RC time constant
t_max = 5.0  # Simulate up to 5 tau
N_u = 50  # IC points (oversampled at t=0)
N_f = 5000  # Collocation points
V0 = 1.0  # Initial voltage

### Step 2: Neural Network
Input: $ t $ (1D). Output: $ V(t) $. 3 hidden layers of 20 neurons (simpler for ODE).

In [ ]:
class PINN_ODE(nn.Module):
    def __init__(self, layers=[1] + [20]*3 + [1]):
        super(PINN_ODE, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
    
    def forward(self, t):
        inp = t  # Single input
        for i, layer in enumerate(self.layers[:-1]):
            inp = torch.tanh(layer(inp))
        V = self.layers[-1](inp)
        return V

### Step 3: Generate Training Data
- IC: Multiple points at t=0 with V=1.
- Collocation: Uniform random t in [0, t_max].

In [ ]:
# IC points (all at t=0)
t_u = torch.zeros((N_u, 1), requires_grad=True).to(device)
V_u = torch.full_like(t_u, V0)

# Collocation points
t_f = torch.rand((N_f, 1), dtype=torch.float32) * t_max
t_f = t_f.requires_grad_(True).to(device)

### Step 4: Physics Loss
ODE residual: $ \frac{dV}{dt} + \frac{V}{\tau} = 0 $.

In [ ]:
def physics_loss(model, t_f, tau):
    V_pred = model(t_f)
    dV_dt = torch.autograd.grad(V_pred, t_f, grad_outputs=torch.ones_like(V_pred),
                                create_graph=True, retain_graph=True)[0]
    f = dV_dt + V_pred / tau
    return torch.mean(f**2)

def total_loss(model, t_u, V_u, t_f, tau, lambda_f=1.0):
    V_pred_u = model(t_u)
    loss_u = torch.mean((V_pred_u - V_u)**2)
    loss_f = physics_loss(model, t_f, tau)
    return loss_u + lambda_f * loss_f

### Step 5: Training
Adam for 2000 epochs, then LBFGS for 500.

In [ ]:
model_ode = PINN_ODE().to(device)
optimizer_adam = torch.optim.Adam(model_ode.parameters(), lr=1e-3)
optimizer_lbfgs = torch.optim.LBFGS(model_ode.parameters(), lr=1.0, max_iter=50)

def closure_ode():
    optimizer_lbfgs.zero_grad()
    loss = total_loss(model_ode, t_u, V_u, t_f, tau)
    loss.backward(retain_graph=True)
    return loss

print("Training RC Circuit PINN...")
for epoch in range(2000):
    optimizer_adam.zero_grad()
    loss = total_loss(model_ode, t_u, V_u, t_f, tau)
    loss.backward()
    optimizer_adam.step()
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

for epoch in range(500):
    optimizer_lbfgs.step(closure_ode)
    loss = closure_ode()
    if epoch % 100 == 0:
        print(f"LBFGS {epoch}, Loss: {loss.item():.6f}")

### Step 6: Evaluation and Plot
Compare to exact solution.

In [ ]:
# Fine grid
t_test = torch.linspace(0, t_max, 100).reshape(-1, 1).to(device)
with torch.no_grad():
    V_pred = model_ode(t_test).cpu()

# Exact
t_exact = t_test.cpu().squeeze()
V_exact = np.exp(-t_exact / tau)

plt.figure(figsize=(8, 5))
plt.plot(t_exact, V_exact, 'b-', label='Exact: $V(t) = e^{-t/\\tau}$')
plt.plot(t_exact, V_pred.squeeze(), 'r--', label='PINN Prediction')
plt.xlabel('Time t (s)')
plt.ylabel('Voltage V(t) (V)')
plt.title('RC Circuit: Capacitor Voltage Decay')
plt.legend()
plt.grid(True)
plt.show()

**Expected Output**: The PINN curve should hug the exponential decay closely (error < 1e-3). Discuss: How does this enforce Kirchhoff? (Residual minimizes current imbalance.)

## Example 2: 1D Heat Diffusion Equation
### Problem Setup
Heat conduction in a 1D rod (length L=1 m), insulated ends (zero flux BCs).

PDE (Fourier's law):
\[
\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial x^2}
\]
- Domain: $ x \in [0, 1] $, $ t \in [0, 0.5] $.
- Initial condition (IC): $ T(x,0) = \sin(\pi x) $ (hot in middle).
- Boundary conditions (BCs): $ \frac{\partial T}{\partial x}(0,t) = \frac{\partial T}{\partial x}(1,t) = 0 $ (Neumann; no heat loss).
- Diffusivity: $ \alpha = 0.1 $ (slow diffusion for visibility).

Exact solution involves Fourier series, but PINNs approximate it mesh-free.

Use:
- **100 points** for IC/BCs.
- **10,000 collocation points** for PDE.

This builds on Burgers' but linear—easier convergence.

### Step 1: Imports and Setup
(Same as before; add:)

In [ ]:
alpha = 0.1  # Thermal diffusivity
x_min, x_max = 0.0, 1.0
t_max_heat = 0.5
N_u_heat = 100
N_f_heat = 10000

### Step 2: Neural Network
Same as Burgers': Input (x,t), 4x50 tanh layers.

(Reuse `PINN` class from original tutorial.)

### Step 3: Generate Data
- IC: t=0, x uniform.
- BCs: x=0 and x=1, t uniform (but enforce flux=0 via derivative in loss).
- Collocation: Random (x,t).

For Neumann BCs, sample points on boundaries and add loss for dT/dx=0.

In [ ]:
def generate_heat_data(N_u, N_f, x_min, x_max, t_max, alpha):
    # IC points
    N_init = N_u // 3
    x_init = torch.linspace(x_min, x_max, N_init).reshape(-1, 1).requires_grad_(True)
    t_init = torch.zeros_like(x_init).requires_grad_(True)
    T_init = torch.sin(np.pi * x_init)
    
    # Left BC: x=0, t uniform (for dT/dx=0)
    N_left = N_u // 3
    x_left = torch.zeros((N_left, 1), requires_grad=True).to(device)
    t_left = torch.rand((N_left, 1)) * t_max
    t_left.requires_grad_(True).to(device)
    
    # Right BC: x=1, t uniform
    N_right = N_u - N_init - N_left
    x_right = torch.ones((N_right, 1), requires_grad=True).to(device)
    t_right = torch.rand((N_right, 1)) * t_max
    t_right.requires_grad_(True).to(device)
    
    # Concat IC + boundaries (no T values for BCs; enforced via derivative)
    x_u = torch.cat([x_init, x_left, x_right], dim=0).to(device)
    t_u = torch.cat([t_init, t_left, t_right], dim=0).to(device)
    T_u = torch.cat([T_init, torch.zeros((N_u - N_init, 1))], dim=0).to(device)  # Dummy for BCs; override in loss
    
    # Collocation
    x_f = torch.rand((N_f, 1)) * (x_max - x_min) + x_min
    t_f = torch.rand((N_f, 1)) * t_max
    x_f.requires_grad_(True).to(device)
    t_f.requires_grad_(True).to(device)
    
    return x_u, t_u, T_u, x_f, t_f

x_u_h, t_u_h, T_u_h, x_f_h, t_f_h = generate_heat_data(N_u_heat, N_f_heat, x_min, x_max, t_max_heat, alpha)

### Step 4: Physics Loss
PDE: $ T_t - \alpha T_{xx} = 0 $.

For BCs: Add MSE on $ T_x $ at boundaries.

In [ ]:
def heat_physics_loss(model, x_f, t_f, alpha, x_u, t_u):
    # PDE residual
    T_pred = model(x_f, t_f)
    T_x = torch.autograd.grad(T_pred, x_f, torch.ones_like(T_pred), create_graph=True, retain_graph=True)[0]
    T_t = torch.autograd.grad(T_pred, t_f, torch.ones_like(T_pred), create_graph=True, retain_graph=True)[0]
    T_xx = torch.autograd.grad(T_x, x_f, torch.ones_like(T_x), create_graph=True, retain_graph=True)[0]
    f_pde = T_t - alpha * T_xx
    loss_pde = torch.mean(f_pde**2)
    
    # Neumann BC loss: dT/dx = 0 at x=0 and x=1
    # Left
    T_left = model(x_u[:N_u_heat//3 + N_init//3], t_u[:N_u_heat//3 + N_init//3])  # Wait, better slice properly
    # Actually, separate BC points
    idx_left = slice(N_init, N_init + N_left)
    idx_right = slice(N_init + N_left, None)
    T_x_left = torch.autograd.grad(model(x_u[idx_left], t_u[idx_left]), x_u[idx_left], 
                                   torch.ones_like(model(x_u[idx_left], t_u[idx_left])), 
                                   create_graph=True)[0]
    T_x_right = torch.autograd.grad(model(x_u[idx_right], t_u[idx_right]), x_u[idx_right], 
                                    torch.ones_like(model(x_u[idx_right], t_u[idx_right])), 
                                    create_graph=True)[0]
    loss_bc = torch.mean(T_x_left**2) + torch.mean(T_x_right**2)
    
    return loss_pde + loss_bc

def heat_total_loss(model, x_u, t_u, T_u, x_f, t_f, alpha):
    # IC loss (only for init points)
    idx_init = slice(0, N_u_heat // 3)
    T_pred_init = model(x_u[idx_init], t_u[idx_init])
    loss_ic = torch.mean((T_pred_init - T_u[idx_init])**2)
    
    # Physics (PDE + BC)
    loss_phys = heat_physics_loss(model, x_f, t_f, alpha, x_u, t_u)
    
    return loss_ic + loss_phys

Wait, small fix: In generate, N_init = N_u//3, etc. For simplicity, in loss, we can compute IC separately.

To make it clean, adjust:

In total_loss:
- loss_u = MSE only on IC points.
- loss_f = PDE on collocation + BC derivative on boundary points.

Yes.

### Step 5: Training
Similar loop.

In [ ]:
model_heat = PINN().to(device)  # From original
optimizer_adam_h = torch.optim.Adam(model_heat.parameters(), lr=1e-3)
optimizer_lbfgs_h = torch.optim.LBFGS(model_heat.parameters(), lr=1.0, max_iter=50)

def closure_heat():
    optimizer_lbfgs_h.zero_grad()
    loss = heat_total_loss(model_heat, x_u_h, t_u_h, T_u_h, x_f_h, t_f_h, alpha)
    loss.backward()
    return loss

# Training (adjust epochs as needed)
for epoch in range(3000):
    optimizer_adam_h.zero_grad()
    loss = heat_total_loss(model_heat, x_u_h, t_u_h, T_u_h, x_f_h, t_f_h, alpha)
    loss.backward()
    optimizer_adam_h.step()
    if epoch % 500 == 0:
        print(f"Heat Epoch {epoch}, Loss: {loss.item():.6f}")

for epoch in range(500):
    optimizer_lbfgs_h.step(closure_heat)
    loss = closure_heat()
    if epoch % 100 == 0:
        print(f"Heat LBFGS {epoch}, Loss: {loss.item():.6f}")

### Step 6: Evaluation and Plot
Contour of T(x,t).

In [ ]:
# Fine grid
x_test_h = torch.linspace(x_min, x_max, 100).reshape(-1, 1).to(device)
t_test_h = torch.linspace(0, t_max_heat, 50).reshape(-1, 1).to(device)
X_h, T_h = torch.meshgrid(x_test_h.squeeze(), t_test_h.squeeze(), indexing='xy')

with torch.no_grad():
    Temp_pred = model_heat(X_h.flatten()[:, None], T_h.flatten()[:, None]).reshape(X_h.shape)

plt.figure(figsize=(10, 4))
contour = plt.contourf(T_h.cpu(), X_h.cpu(), Temp_pred.cpu(), levels=50, cmap='hot')
plt.xlabel('Time t')
plt.ylabel('Position x')
plt.title("PINN Solution: Temperature T(x,t) in 1D Rod")
plt.colorbar(contour)
plt.tight_layout()
plt.show()

# IC comparison
idx_init_h = slice(0, N_u_heat // 3)
T_init_pred = model_heat(x_u_h[idx_init_h], t_u_h[idx_init_h]).detach().cpu()
plt.figure(figsize=(8, 4))
x_init_plot = x_u_h[idx_init_h].cpu().squeeze()
plt.plot(x_init_plot, torch.sin(np.pi * x_init_plot.cpu()), 'b-', label='True IC')
plt.plot(x_init_plot, T_init_pred.squeeze(), 'r--', label='PINN IC')
plt.xlabel('x')
plt.ylabel('T(x,0)')
plt.title('Initial Temperature Profile')
plt.legend()
plt.grid(True)
plt.show()

**Expected Output**: Heat diffuses from center, flattening over time. Contours smooth; error small.

## Wrapping Up
These examples bridge familiar physics to ML: RC shows ODEs (circuits class), heat PDEs (thermo class). Encourage students to:
- Modify ICs (e.g., step function for heat).
- Add noise to "data" for inverse problems (estimate α from measurements).
- Compare runtime to finite differences.

In [ ]:
# Physics-Informed Neural Networks (PINNs) in PyTorch: Familiar Examples Tutorial

Absolutely! Great idea—students often find electrical circuits and heat transfer more intuitive than fluid dynamics. We'll adapt the PINN framework to two classic problems they're likely familiar with from intro physics/engineering courses:

1. **Kirchhoff's Voltage Law (KVL) for an RC Circuit**: This leads to a simple **ordinary differential equation (ODE)** for the capacitor voltage decay. It's a 1D time-domain problem.
2. **1D Heat Diffusion Equation**: The classic **heat equation**, a linear PDE modeling temperature evolution in a rod.

These showcase PINNs for both ODEs (easier entry) and PDEs. We'll keep the structure similar to the Burgers' tutorial: PyTorch-based, with automatic differentiation, Adam + LBFGS optimization, and visualizations. Each section is self-contained—run them independently in a Jupyter notebook.

Assumptions: Students know basic PyTorch, ODEs/PDEs, and circuits/heat transfer. Runtime: ~1-2 minutes per example on CPU.

## Quick PINN Recap
PINNs approximate solutions $ u(\mathbf{x}) $ (e.g., voltage or temperature) with a neural net, trained to minimize:
- **Data loss**: Matches initial/boundary conditions.
- **Physics loss**: PDE/ODE residual ≈ 0 at random "collocation" points.

## Example 1: RC Circuit ODE (Kirchhoff's Law)
### Problem Setup
Consider a series RC circuit with resistor $ R $ and capacitor $ C $, charged to initial voltage $ V_0 = 1 $ V, then discharged (switch opens at $ t=0 $).

By KVL: Voltage drop across R + C = 0 (no source), so:
\[
\frac{dV}{dt} = -\frac{V}{RC} = -\frac{V}{\tau}
\]
- Time domain: $ t \in [0, 5\tau] $, where $ \tau = RC = 1 $ s (normalize for simplicity).
- Initial condition (IC): $ V(0) = 1 $.
- Exact solution: $ V(t) = e^{-t/\tau} $ (exponential decay).

We'll use:
- **50 IC points** (just t=0, but duplicated for stability; treat as "data").
- **5,000 collocation points** in time for ODE residual.

This teaches ODE solving—PINNs shine for stiff/nonlinear ODEs, but works great here.

### Step 1: Imports and Setup
```python
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Problem parameters
tau = 1.0  # RC time constant
t_max = 5.0  # Simulate up to 5 tau
N_u = 50  # IC points (oversampled at t=0)
N_f = 5000  # Collocation points
V0 = 1.0  # Initial voltage
```

### Step 2: Neural Network
Input: $ t $ (1D). Output: $ V(t) $. 3 hidden layers of 20 neurons (simpler for ODE).

```python
class PINN_ODE(nn.Module):
    def __init__(self, layers=[1] + [20]*3 + [1]):
        super(PINN_ODE, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
    
    def forward(self, t):
        inp = t  # Single input
        for i, layer in enumerate(self.layers[:-1]):
            inp = torch.tanh(layer(inp))
        V = self.layers[-1](inp)
        return V
```

### Step 3: Generate Training Data
- IC: Multiple points at t=0 with V=1.
- Collocation: Uniform random t in [0, t_max].

```python
# IC points (all at t=0)
t_u = torch.zeros((N_u, 1), requires_grad=True).to(device)
V_u = torch.full_like(t_u, V0)

# Collocation points
t_f = torch.rand((N_f, 1), dtype=torch.float32) * t_max
t_f = t_f.requires_grad_(True).to(device)
```

### Step 4: Physics Loss
ODE residual: $ \frac{dV}{dt} + \frac{V}{\tau} = 0 $.

```python
def physics_loss(model, t_f, tau):
    V_pred = model(t_f)
    dV_dt = torch.autograd.grad(V_pred, t_f, grad_outputs=torch.ones_like(V_pred),
                                create_graph=True, retain_graph=True)[0]
    f = dV_dt + V_pred / tau
    return torch.mean(f**2)

def total_loss(model, t_u, V_u, t_f, tau, lambda_f=1.0):
    V_pred_u = model(t_u)
    loss_u = torch.mean((V_pred_u - V_u)**2)
    loss_f = physics_loss(model, t_f, tau)
    return loss_u + lambda_f * loss_f
```

### Step 5: Training
Adam for 2000 epochs, then LBFGS for 500.

```python
model_ode = PINN_ODE().to(device)
optimizer_adam = torch.optim.Adam(model_ode.parameters(), lr=1e-3)
optimizer_lbfgs = torch.optim.LBFGS(model_ode.parameters(), lr=1.0, max_iter=50)

def closure_ode():
    optimizer_lbfgs.zero_grad()
    loss = total_loss(model_ode, t_u, V_u, t_f, tau)
    loss.backward(retain_graph=True)
    return loss

print("Training RC Circuit PINN...")
for epoch in range(2000):
    optimizer_adam.zero_grad()
    loss = total_loss(model_ode, t_u, V_u, t_f, tau)
    loss.backward()
    optimizer_adam.step()
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

for epoch in range(500):
    optimizer_lbfgs.step(closure_ode)
    loss = closure_ode()
    if epoch % 100 == 0:
        print(f"LBFGS {epoch}, Loss: {loss.item():.6f}")
```

### Step 6: Evaluation and Plot
Compare to exact solution.

```python
# Fine grid
t_test = torch.linspace(0, t_max, 100).reshape(-1, 1).to(device)
with torch.no_grad():
    V_pred = model_ode(t_test).cpu()

# Exact
t_exact = t_test.cpu().squeeze()
V_exact = np.exp(-t_exact / tau)

plt.figure(figsize=(8, 5))
plt.plot(t_exact, V_exact, 'b-', label='Exact: $V(t) = e^{-t/\\tau}$')
plt.plot(t_exact, V_pred.squeeze(), 'r--', label='PINN Prediction')
plt.xlabel('Time t (s)')
plt.ylabel('Voltage V(t) (V)')
plt.title('RC Circuit: Capacitor Voltage Decay')
plt.legend()
plt.grid(True)
plt.show()
```

**Expected Output**: The PINN curve should hug the exponential decay closely (error < 1e-3). Discuss: How does this enforce Kirchhoff? (Residual minimizes current imbalance.)

## Example 2: 1D Heat Diffusion Equation
### Problem Setup
Heat conduction in a 1D rod (length L=1 m), insulated ends (zero flux BCs).

PDE (Fourier's law):
\[
\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial x^2}
\]
- Domain: $ x \in [0, 1] $, $ t \in [0, 0.5] $.
- Initial condition (IC): $ T(x,0) = \sin(\pi x) $ (hot in middle).
- Boundary conditions (BCs): $ \frac{\partial T}{\partial x}(0,t) = \frac{\partial T}{\partial x}(1,t) = 0 $ (Neumann; no heat loss).
- Diffusivity: $ \alpha = 0.1 $ (slow diffusion for visibility).

Exact solution involves Fourier series, but PINNs approximate it mesh-free.

Use:
- **100 points** for IC/BCs.
- **10,000 collocation points** for PDE.

This builds on Burgers' but linear—easier convergence.

### Step 1: Imports and Setup
(Same as before; add:)
```python
alpha = 0.1  # Thermal diffusivity
x_min, x_max = 0.0, 1.0
t_max_heat = 0.5
N_u_heat = 100
N_f_heat = 10000
```

### Step 2: Neural Network
Same as Burgers': Input (x,t), 4x50 tanh layers.

(Reuse `PINN` class from original tutorial.)

### Step 3: Generate Data
- IC: t=0, x uniform.
- BCs: x=0 and x=1, t uniform (but enforce flux=0 via derivative in loss).
- Collocation: Random (x,t).

For Neumann BCs, sample points on boundaries and add loss for dT/dx=0.

```python
def generate_heat_data(N_u, N_f, x_min, x_max, t_max, alpha):
    # IC points
    N_init = N_u // 3
    x_init = torch.linspace(x_min, x_max, N_init).reshape(-1, 1).requires_grad_(True)
    t_init = torch.zeros_like(x_init).requires_grad_(True)
    T_init = torch.sin(np.pi * x_init)
    
    # Left BC: x=0, t uniform (for dT/dx=0)
    N_left = N_u // 3
    x_left = torch.zeros((N_left, 1), requires_grad=True).to(device)
    t_left = torch.rand((N_left, 1)) * t_max
    t_left.requires_grad_(True).to(device)
    
    # Right BC: x=1, t uniform
    N_right = N_u - N_init - N_left
    x_right = torch.ones((N_right, 1), requires_grad=True).to(device)
    t_right = torch.rand((N_right, 1)) * t_max
    t_right.requires_grad_(True).to(device)
    
    # Concat IC + boundaries (no T values for BCs; enforced via derivative)
    x_u = torch.cat([x_init, x_left, x_right], dim=0).to(device)
    t_u = torch.cat([t_init, t_left, t_right], dim=0).to(device)
    T_u = torch.cat([T_init, torch.zeros((N_u - N_init, 1))], dim=0).to(device)  # Dummy for BCs; override in loss
    
    # Collocation
    x_f = torch.rand((N_f, 1)) * (x_max - x_min) + x_min
    t_f = torch.rand((N_f, 1)) * t_max
    x_f.requires_grad_(True).to(device)
    t_f.requires_grad_(True).to(device)
    
    return x_u, t_u, T_u, x_f, t_f

x_u_h, t_u_h, T_u_h, x_f_h, t_f_h = generate_heat_data(N_u_heat, N_f_heat, x_min, x_max, t_max_heat, alpha)
```

### Step 4: Physics Loss
PDE: $ T_t - \alpha T_{xx} = 0 $.

For BCs: Add MSE on $ T_x $ at boundaries.

```python
def heat_physics_loss(model, x_f, t_f, alpha, x_u, t_u):
    # PDE residual
    T_pred = model(x_f, t_f)
    T_x = torch.autograd.grad(T_pred, x_f, torch.ones_like(T_pred), create_graph=True, retain_graph=True)[0]
    T_t = torch.autograd.grad(T_pred, t_f, torch.ones_like(T_pred), create_graph=True, retain_graph=True)[0]
    T_xx = torch.autograd.grad(T_x, x_f, torch.ones_like(T_x), create_graph=True, retain_graph=True)[0]
    f_pde = T_t - alpha * T_xx
    loss_pde = torch.mean(f_pde**2)
    
    # Neumann BC loss: dT/dx = 0 at x=0 and x=1
    # Left
    T_left = model(x_u[:N_u_heat//3 + N_init//3], t_u[:N_u_heat//3 + N_init//3])  # Wait, better slice properly
    # Actually, separate BC points
    idx_left = slice(N_init, N_init + N_left)
    idx_right = slice(N_init + N_left, None)
    T_x_left = torch.autograd.grad(model(x_u[idx_left], t_u[idx_left]), x_u[idx_left], 
                                   torch.ones_like(model(x_u[idx_left], t_u[idx_left])), 
                                   create_graph=True)[0]
    T_x_right = torch.autograd.grad(model(x_u[idx_right], t_u[idx_right]), x_u[idx_right], 
                                    torch.ones_like(model(x_u[idx_right], t_u[idx_right])), 
                                    create_graph=True)[0]
    loss_bc = torch.mean(T_x_left**2) + torch.mean(T_x_right**2)
    
    return loss_pde + loss_bc

def heat_total_loss(model, x_u, t_u, T_u, x_f, t_f, alpha):
    # IC loss (only for init points)
    idx_init = slice(0, N_u_heat // 3)
    T_pred_init = model(x_u[idx_init], t_u[idx_init])
    loss_ic = torch.mean((T_pred_init - T_u[idx_init])**2)
    
    # Physics (PDE + BC)
    loss_phys = heat_physics_loss(model, x_f, t_f, alpha, x_u, t_u)
    
    return loss_ic + loss_phys
```

Wait, small fix: In generate, N_init = N_u//3, etc. For simplicity, in loss, we can compute IC separately.

To make it clean, adjust:

In total_loss:
- loss_u = MSE only on IC points.
- loss_f = PDE on collocation + BC derivative on boundary points.

Yes.

### Step 5: Training
Similar loop.

```python
model_heat = PINN().to(device)  # From original
optimizer_adam_h = torch.optim.Adam(model_heat.parameters(), lr=1e-3)
optimizer_lbfgs_h = torch.optim.LBFGS(model_heat.parameters(), lr=1.0, max_iter=50)

def closure_heat():
    optimizer_lbfgs_h.zero_grad()
    loss = heat_total_loss(model_heat, x_u_h, t_u_h, T_u_h, x_f_h, t_f_h, alpha)
    loss.backward()
    return loss

# Training (adjust epochs as needed)
for epoch in range(3000):
    optimizer_adam_h.zero_grad()
    loss = heat_total_loss(model_heat, x_u_h, t_u_h, T_u_h, x_f_h, t_f_h, alpha)
    loss.backward()
    optimizer_adam_h.step()
    if epoch % 500 == 0:
        print(f"Heat Epoch {epoch}, Loss: {loss.item():.6f}")

for epoch in range(500):
    optimizer_lbfgs_h.step(closure_heat)
    loss = closure_heat()
    if epoch % 100 == 0:
        print(f"Heat LBFGS {epoch}, Loss: {loss.item():.6f}")
```

### Step 6: Evaluation and Plot
Contour of T(x,t).

```python
# Fine grid
x_test_h = torch.linspace(x_min, x_max, 100).reshape(-1, 1).to(device)
t_test_h = torch.linspace(0, t_max_heat, 50).reshape(-1, 1).to(device)
X_h, T_h = torch.meshgrid(x_test_h.squeeze(), t_test_h.squeeze(), indexing='xy')

with torch.no_grad():
    Temp_pred = model_heat(X_h.flatten()[:, None], T_h.flatten()[:, None]).reshape(X_h.shape)

plt.figure(figsize=(10, 4))
contour = plt.contourf(T_h.cpu(), X_h.cpu(), Temp_pred.cpu(), levels=50, cmap='hot')
plt.xlabel('Time t')
plt.ylabel('Position x')
plt.title("PINN Solution: Temperature T(x,t) in 1D Rod")
plt.colorbar(contour)
plt.tight_layout()
plt.show()

# IC comparison
idx_init_h = slice(0, N_u_heat // 3)
T_init_pred = model_heat(x_u_h[idx_init_h], t_u_h[idx_init_h]).detach().cpu()
plt.figure(figsize=(8, 4))
x_init_plot = x_u_h[idx_init_h].cpu().squeeze()
plt.plot(x_init_plot, torch.sin(np.pi * x_init_plot.cpu()), 'b-', label='True IC')
plt.plot(x_init_plot, T_init_pred.squeeze(), 'r--', label='PINN IC')
plt.xlabel('x')
plt.ylabel('T(x,0)')
plt.title('Initial Temperature Profile')
plt.legend()
plt.grid(True)
plt.show()
```

**Expected Output**: Heat diffuses from center, flattening over time. Contours smooth; error small.

## Wrapping Up
These examples bridge familiar physics to ML: RC shows ODEs (circuits class), heat PDEs (thermo class). Encourage students to:
- Modify ICs (e.g., step function for heat).
- Add noise to "data" for inverse problems (estimate α from measurements).
- Compare runtime to finite differences.

Full code in one notebook? Paste sections together. Questions? Let's tweak! 🔧

SyntaxError: invalid character '—' (U+2014) (1666217392.py, line 3)